# CERTUTIL hunt

This notebook helps to collect all cmd (cmd.exe) and (certutil.exe) process executions in order to find suspicious activity.

This example demonstrates how to find suspicious executions that are downloaded by using certutil.exe, and then using certutil.exe to attack.

In [1]:
from pyclient.stix_shifter_dataframe import StixShifterDataFrame
from dateutil import parser
import re
import pandas as pd

In [2]:
carbon_black_stix_bundle_1 = 'https://raw.githubusercontent.com/opencybersecurityalliance/stix-shifter/master/data/cybox/carbon_black/carbon_black_observable.json'
sb_config_1 = {
    'translation_module': 'stix_bundle',
    'transmission_module': 'stix_bundle',
    'connection': {
        "host": carbon_black_stix_bundle_1,
        "port": 443
    },
    'configuration': {
        "auth": {
            "username": None,
            "password": None
        }
    },
    'data_source': '{"type": "identity", "id": "identity--3532c56d-ea72-48be-a2ad-1a53f4c9c6d3", "name": "stix_boundle", "identity_class": "events"}'
}

In [3]:
carbon_black_stix_bundle_2 = 'https://raw.githubusercontent.com/opencybersecurityalliance/stix-shifter/develop/data/cybox/carbon_black/cb_observed_156.json'
sb_config_2 = {
    'translation_module': 'stix_bundle',
    'transmission_module': 'stix_bundle',
    'connection': {
        "host": carbon_black_stix_bundle_2,
        "port": 443
    },
    'configuration': {
        "auth": {
            "username": None,
            "password": None
        }
    },
    'data_source': '{"type": "identity", "id": "identity--3532c56d-ea72-48be-a2ad-1a53f4c9c6d3", "name": "stix_boundle", "identity_class": "events"}'
}

In [4]:
def get_duration(duration):
    days, seconds = duration.days, duration.seconds
    hours = seconds // 3600
    minutes = (seconds % 3600) // 60
    seconds = seconds % 60
    return f"{days}d {hours}h {minutes}m {seconds}.{duration.microseconds//1000}s"

In [5]:
def defang(url):
    return re.sub('http', 'hxxp', url)

# Fetch process data that are spawn by cmd

In [6]:
ssdf = StixShifterDataFrame()
ssdf.add_config('cb_stix_bundle_1', sb_config_1)
ssdf.add_config('cb_stix_bundle_2', sb_config_2)


# stix-shifter uses STIX patterning as its query language
# See http://docs.oasis-open.org/cti/stix/v2.0/cs01/part5-stix-patterning/stix-v2.0-cs01-part5-stix-patterning.html
cmd_query = "[process:name = 'cmd.exe']"
df = ssdf.search_df(query=cmd_query, config_names=['cb_stix_bundle_1', 'cb_stix_bundle_2'])

In [7]:
df['first_observed'] = pd.to_datetime(df['first_observed'], infer_datetime_format=True, utc=True)
df['last_observed'] = pd.to_datetime(df['last_observed'], infer_datetime_format=True, utc=True)
df['duration'] = df['last_observed'] - df['first_observed']
df['duration'] = df['duration'].map(lambda dur: get_duration(dur))

In [8]:
df['first_observed'] = pd.to_datetime(df['first_observed'], infer_datetime_format=True, utc=True)
df['last_observed'] = pd.to_datetime(df['last_observed'], infer_datetime_format=True, utc=True)
df['duration'] = df['last_observed'] - df['first_observed']
df['duration'] = df['duration'].map(lambda dur: get_duration(dur))

In [9]:
df.head()

# Find suspicious command line

In [10]:
# Use a regex to find suspicious certutil usage
susp = df[df['process:command_line'].str.contains(r'certutil.*[0-9a-zA-Z_-]*\.(exe|dat)')]

# Look at the matches (defang any URLs in there since jupyter makes them clickable!)
list(map(defang, susp['process:command_line'].head().to_list()))

# Attack steps

In [11]:
fields = ['first_observed', 'last_observed', 'duration',
    'process:name', 'process:pid', 
    'process:binary_ref.name', 'process:parent_ref.name', 
    'network-traffic:dst_ref.value', 'network-traffic:src_ref.value', 
    'process:command_line', 'user-account:user_id'
]

df[fields].sort_values(by=['first_observed'])

In this notebook, we finally found that this is a APT attack , ```c64.exe f64.data "9839D7F1A0 -m" ```
Ref: https://www.fireeye.com/blog/threat-research/2019/08/game-over-detecting-and-stopping-an-apt41-operation.html